# Stock Market News Sentiment
This project looks into classifying the sentiment of stock market news and tweets into a bullish or bearish outlook. The data is from [Kaggle: Stock-Market Sentiment Dataset](https://www.kaggle.com/datasets/yash612/stockmarket-sentiment-dataset).

The data is broken up into two columns: `Text` that is a block of text and `Sentiment` that is a value between -1 and 1 value, meaning -1 is negative sentiment, 0 is neutral, and 1 is positive sentiment. 

# Set up
Import the necessary dependencies and load the data

In [ ]:
import pandas as pd
import numpy as np
from string import punctuation
import pyprind
import re

import nltk
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import RNN, Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('data.csv')

# EDA & Preprocessing

## EDA

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.nunique()

In [ ]:
data['Num_Words'] = data['Text'].apply(lambda x: len(x.split()))

max_words = data['Num_Words'].max()
min_words = data['Num_Words'].min()
avg_words = data['Num_Words'].mean()

print(f"Max words: {max_words}\nMin words: {min_words},\nAvg words: {avg_words}")

In [ ]:
sentiment_counts = data['Sentiment'].value_counts()
sentiment_counts

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(x='Sentiment', data=data)
plt.title('Distribution of Labels in Training Set')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
%matplotlib inline

avg_words_per_category = data.groupby('Sentiment')['Num_Words'].mean()

avg_words_per_category.plot(kind='bar')
plt.xlabel('Sentiment')
plt.ylabel('Average Number of Words')
plt.title('Average Words per Sentiment')

plt.xticks(rotation=90)

plt.show()

## Preprocessing
We need to preprocess the data to make it readable for the model. 

**Steps**:
1. We need to drop all duplicates 

2. We need to remove all the stop words in the text using `nltk`'s stopwords.

3. We need to remove HTML tags, URLs, special characters, and convert to lowercase. 

4. We need to find unique words in dataset (using `Counter()`) to count each word's occurence and convert text to integers. Then we will create a mapping to map each unique word to integer. 

5. We need to make the sequences same length. The sequences that don't meet that length will be padded with '0's' while the longer ones will be cut. We will define same-length sequences by:
    - If sequence length < 200: left-pad with zeros
    - If sequence length > 200: use the last 200 elements

6. Use `train_test_split` to get an 80/20 split of the data

In [ ]:
data = data.drop_duplicates(subset=['Text'], ignore_index=True)

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
data['Text'] = data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

In [ ]:
data['Text'] = data['Text'].apply(lambda x: re.sub(r'<.*?>', '', x))
data['Text'] = data['Text'].apply(lambda x: re.sub(r'http\S+', '', x))
data['Text'] = data['Text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
data['Text'] = data['Text'].apply(lambda x: x.lower())

In [ ]:
counts = Counter()
pbar = pyprind.ProgBar(len(data['Text']),
                       title='Counting words occurences')
for i, sentiment in enumerate(data['Text']):
    text = ''.join([c if c not in punctuation else ' '+c+' ' for c in sentiment]).lower()
    data.loc[i,'Text'] = text
    pbar.update()
    counts.update(text.split())


word_counts = sorted(counts, key=counts.get, reverse=True)
print(word_counts[:5])
word_to_int = {word: ii for ii, word in enumerate(word_counts, 1)}

mapped_sentiment = []
pbar = pyprind.ProgBar(len(data['Text']),
                       title='Map sentiments to ints')
for sentiment in data['Text']:
    mapped_sentiment.append([word_to_int[word] for word in sentiment.split()])
    pbar.update()

In [ ]:
sequence_length = 200  
sequences = np.zeros((len(mapped_sentiment), sequence_length), dtype=int)
for i, row in enumerate(mapped_sentiment):
    sentiment_arr = np.array(row)
    sequences[i, -len(row):] = sentiment_arr[-sequence_length:]

In [ ]:
X = sequences
y = data['Sentiment'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
# Define helper function that breaks given datset into chunks and returns a generator to iterate through mini-batches
np.random.seed(123) # for reproducibility

# Define a fxn to generate mini-batches
def create_batch_generator(x, y=None, batch_size=64):
    n_batches = len(x)//batch_size
    x = x[:n_batches*batch_size]
    if y is not None:
        y = y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        if y is not None:
            yield x[ii:ii+batch_size], y[ii:ii+batch_size]
        else:
            yield x[ii:ii+batch_size]

# Model Architecture

The `SentimentRNN` class implements a recurrent neural network (RNN) for sentiment analysis using TensorFlow and Keras. It utilizes an **embedding layer** followed by **LSTM** (Long Short-Term Memory) layers with bidirectional processing to capture sequential dependencies effectively. Dropout regularization is applied to mitigate overfitting, and the final output layer uses a sigmoid activation function for binary classification. This architecture leverages the strengths of LSTM networks for handling sequential data and is suitable for tasks like sentiment analysis where understanding the context and dependencies among words is crucial for accurate classification:

**Embedding Layer:**
  - **Input:** 
    - Dimension of the vocabulary (`n_words`)
    - Input sequence length (`seq_len`)
  - **Functionality:** 
    - Maps each word index to a dense vector representation (`embed_size`)
   
**LSTM Layers:**
  - **Bidirectional LSTM:**
    - **Units:** `lstm_size`
    - **Number of Layers:** `num_layers`
    - **Return Sequences:** True for all layers except the last one
    - **Functionality:** 
      - Captures forward and backward context of the input sequences
      - Helps in understanding context from both past and future states

**Dropout Layers:**
  - **Usage:** 
    - Applied after each LSTM layer (`Dropout(0.5)`)
    - Reduces overfitting by randomly setting a fraction of inputs to zero during training
   
**Dense Output Layer:**
  - **Activation:** Sigmoid
  - **Functionality:** 
    - Outputs a single probability score for binary sentiment classification

**Optimizer and Loss Function:**
  - **Optimizer:** Adam with a learning rate (`learning_rate`)
  - **Loss Function:** Binary Cross-Entropy
  - **Metrics:** Accuracy for evaluation during training
   
**Training:**
  - Input: `X_train` (numpy array of shape `(num_samples, seq_len)`)
  - Labels: `y_train` (numpy array of shape `(num_samples,)`)
  - Trains the model using `num_epochs` with a batch size of `batch_size`
  - Validates on 10% of the training data (`validation_split=0.1`)

**Prediction:**
  - Input: `X_data` (numpy array of shape `(num_samples, seq_len)`)
  - Returns either predicted labels (`numpy.ndarray` of integers) or probabilities (`numpy.ndarray` of floats) based on the `return_proba` parameter

In [ ]:
class SentimentRNN:
    """
    A Recurrent Neural Network model for sentiment analysis using TensorFlow and Keras.

    Methods:
        build_model():
            Builds the RNN model using TensorFlow and Keras.
        
        train(X_train, y_train, num_epochs):
            Trains the RNN model on the provided training data.

        predict(X_data, return_proba=False):
            Makes predictions on new data.
    """
    def __init__(self, n_words, seq_len=200,
                 lstm_size=256, num_layers=1, batch_size=64,
                 learning_rate=0.0001, embed_size=200):
        """
        Initializes the SentimentRNN object with specified parameters.

        Args:
            n_words (int): Number of words in the vocabulary.
            seq_len (int, optional): Length of input sequences (default is 200).
            lstm_size (int, optional): Size of LSTM units (default is 256).
            num_layers (int, optional): Number of LSTM layers (default is 1).
            batch_size (int, optional): Batch size for training (default is 64).
            learning_rate (float, optional): Learning rate for the optimizer (default is 0.0001).
            embed_size (int, optional): Size of word embeddings (default is 200).
        """
        self.n_words = n_words
        self.seq_len = seq_len
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.embed_size = embed_size

        self.build_model()

    def build_model(self):
        """
        Builds the LSTM-based sentiment analysis model using tf.keras.Sequential.
        Embedding layer, LSTM layers (with dropout), and Dense output layer are added.
        Adam optimizer is used with binary cross-entropy loss.
        """
        self.model = Sequential()
        self.model.add(Embedding(input_dim=self.n_words, output_dim=self.embed_size, input_length=self.seq_len))
        for _ in range(self.num_layers):
            self.model.add(Bidirectional(LSTM(self.lstm_size, return_sequences=True)))
            self.model.add(Dropout(0.5))
        self.model.add(Bidirectional(LSTM(self.lstm_size)))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(1, activation='sigmoid'))

        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
                           loss='binary_crossentropy',
                           metrics=['accuracy'])
        
        # self.model.summary()

    def train(self, X_train, y_train, num_epochs):
        """
        Trains the sentiment analysis model on the provided training data.

        Args:
            X_train (numpy.ndarray): Training input data of shape (num_samples, seq_len).
            y_train (numpy.ndarray): Training labels of shape (num_samples,).
            num_epochs (int): Number of epochs to train the model.
        """
        self.model.fit(X_train, y_train, epochs=num_epochs, batch_size=self.batch_size, validation_split=0.1)

    def predict(self, X_data, return_proba=False):
        """
        Predicts sentiment labels or probabilities for input data.

        Args:
            X_data (numpy.ndarray): Input data of shape (num_samples, seq_len).
            return_proba (bool, optional): If True, returns predicted probabilities; otherwise, returns labels (default is False).

        Returns:
            numpy.ndarray: Predicted labels (if return_proba=False) or probabilities (if return_proba=True).
        """
        predictions = self.model.predict(X_data, batch_size=self.batch_size)
        if return_proba:
            return predictions
        else:
            return (predictions > 0.5).astype(np.int32)

In [ ]:
# Check the maximum index value in X_train and X_test
n_words = max(list(word_to_int.values())) + 1

# Update the n_words parameter in the SentimentRNN constructor
rnn = SentimentRNN(n_words=n_words)

## Train Model

In [ ]:
rnn.train(X_train, y_train, num_epochs=5)

In [ ]:
preds = rnn.predict(X_test)

In [ ]:
accuracy = np.mean(preds == y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

## Hyperparameter Tuning
Let's see if we can increase the accuracy of this model. We will use GridSearch, but first, we need to create a wrapper for our model in order to use it. 

We will be working on the following hyperparameters:
- `seq_len`: Affects how much contextual information the model can capture from each input text
- `lstm_size`: Directly impacts the model's capacity to learn complex patterns and dependencies within the data
- `num_layers`: Allows the model to learn hierarchical representations of text data, potentially improving its ability to understand nuanced sentiment expressions
- `batch_size`: Affects the gradient update dynamics and training stability
- `learning_rate`: Controls how much to update the model in response to estimated gradients

In [ ]:
class SentimentRNNWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, n_words, seq_len=200, lstm_size=256, num_layers=1,
                 batch_size=64, learning_rate=0.0001, embed_size=200):
        self.n_words = n_words
        self.seq_len = seq_len
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.embed_size = embed_size
        self.model = None

    def fit(self, X, y, num_epochs=5):
        """Fit method for GridSearchCV compatibility.

        Args:
            X: X training data
            y: y training data
            num_epochs (int, optional): _description_. Defaults to 5.

        Returns:
            _type_: _description_
        """
        X, y = check_X_y(X, y, estimator=self, dtype=None)
        
        self.model = SentimentRNN(
            n_words=self.n_words,
            seq_len=self.seq_len,
            lstm_size=self.lstm_size,
            num_layers=self.num_layers,
            batch_size=self.batch_size,
            learning_rate=self.learning_rate,
            embed_size=self.embed_size
        )
        
        self.model.train(X, y, num_epochs=num_epochs)
        return self
    
    def predict(self, X):
        """Predict method for GridSearchCV compatibility

        Args:
            X: Assumes X is the test data

        Returns:
            predictions: Predictions made by the model
        """
        return self.model.predict(X)

    def get_params(self, deep=True):
        """Get parameters for the estimator. This is for GridSearchCV compatibility.

        Args:
            deep (bool, optional): Controls the depth of the attributes that are included in the returned dictionary of parameters. Defaults to True.

        Returns:
            params: Dictionary of parameters
        """
        return {
            'n_words': self.n_words,
            'seq_len': self.seq_len,
            'lstm_size': self.lstm_size,
            'num_layers': self.num_layers,
            'batch_size': self.batch_size,
            'learning_rate': self.learning_rate,
            'embed_size': self.embed_size
        }

    def set_params(self, **params):
        """Set the parameters of the estimator. This is for GridSearchCV compatibility.

        Returns:
            params: Sets the dictionary of parameters
        """
        for param, value in params.items():
            setattr(self, param, value)
        return self


In [ ]:
pipeline = Pipeline([
    ('rnn', SentimentRNNWrapper(n_words=n_words)),
])

# Define parameters for grid search
parameters = {
    'rnn__seq_len': [100, 200, 300],
    'rnn__lstm_size': [128, 256, 512],
    'rnn__num_layers': [1, 2, 3],
    'rnn__batch_size': [32, 64, 128],
    'rnn__learning_rate': [0.001, 0.0001],
}

In [ ]:
grid_search = GridSearchCV(pipeline, parameters, cv=3, verbose=1, scoring='accuracy')

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Test Model
Now it's time to test the best model combination. 

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
predictions = best_model.predict(X_test)

In [ ]:
best_accuracy = np.mean(predictions == y_test)
print(f"Test accuracy: {best_accuracy * 100:.2f}%")

# Conclusion
In this project, we developed a Sentiment Analysis model using a Recurrent Neural Network (RNN) architecture implemented with TensorFlow and Keras. The goal was to classify the sentiment of text data into positive or negative categories. Here’s a summary of our approach and findings:

1. **Model Architecture**: We implemented an RNN with LSTM layers, which are well-suited for sequential data like text due to their ability to capture dependencies over time. The model included embedding layers for text representation, multiple LSTM layers for learning hierarchical features, and dropout layers for regularization.

2. **Hyperparameter Tuning**: We utilized GridSearchCV to systematically search through different combinations of hyperparameters such as sequence length (`seq_len`), LSTM size (`lstm_size`), number of layers (`num_layers`), batch size (`batch_size`), and learning rate (learning_rate). The goal was to optimize model performance by finding the best configuration for these parameters.

3. **Training and Evaluation**: The model was trained on a dataset containing labeled text data for sentiment analysis. We evaluated the model’s performance using metrics like accuracy to measure its ability to correctly predict sentiment labels on unseen data.

4. **Performance**: Through iterative tuning and evaluation, we observed improvements in the model’s accuracy as we optimized hyperparameters. This process allowed us to fine-tune the model to achieve better performance in sentiment classification tasks.

5. **Challenges and Considerations**: During the project, we encountered challenges such as balancing model complexity with computational resources and managing overfitting. Regularization techniques like dropout and careful selection of hyperparameters were crucial in addressing these challenges.

6. **Future Directions**: Moving forward, further enhancements could include exploring more advanced RNN variants (e.g., GRU, bi-directional LSTM), incorporating pre-trained word embeddings (e.g., Word2Vec, GloVe), or leveraging transfer learning techniques from larger language models (e.g., BERT, GPT). Additionally, expanding the dataset size and diversity could help generalize the model’s ability to handle different types of sentiment analysis tasks and domains.

In conclusion, this project demonstrated the effectiveness of RNN-based architectures for sentiment analysis tasks and highlighted the importance of hyperparameter tuning in optimizing model performance. By continually refining and expanding upon these techniques, we can advance the state-of-the-art in natural language processing applications, particularly in sentiment analysis and related domains.